In [2]:
# TU PHAM
# Questions 1, 2, 4
text = read("federalistpapers.txt", String)
essays = split(text, r"FEDERALIST No.\ \d+")
popfirst!(essays)
authors = Array{String}(undef, length(essays))
for i in eachindex(essays)
    essayParts = split(essays[i], "PUBLIUS")
    essayParts = split(essayParts[1], "To the People of the State of New York")
    firstPart = split(essayParts[1], "\r\n\r\n")
    if(i in 49:58 || i in 62:63)
        authors[i] = "Unknown"
    else
        authors[i] = replace(firstPart[end - 1], "\r\n" => "")
    end
    secondPart = essayParts[2]
    essays[i] = string(secondPart[6:end], "PUBLIUS")
end


# Question 3
function separateIntoParagraphs(essays)
    array = Array{String}[]
    for i in eachindex(essays)
        paragraphs = split(essays[i], "\r\n\r\n")
        for j in eachindex(paragraphs)
            paragraphs[j] = replace(paragraphs[j], "\r\n" => " ")
        end
        push!(array, paragraphs)
    end
    return array
end

array = separateIntoParagraphs(essays)

# Questions 5, 6, 7, 8
function editText(text)
    editedText = lowercase(text)
    editedText = replace(editedText, r"\(\d+\)" => "")
    editedText = replace(editedText, r"\d+" => "")
    editedText = replace(editedText, "\r\n" => " ")
    editedText = replace(editedText, r"[,:;'-]"=> "")
    return editedText
end

function editEssays(essays)
    for i in eachindex(essays)
        essays[i] = editText(essays[i])
    end
    return essays
end

function editParagraphs(array)
    for i in eachindex(array)
        for j in eachindex(array[i])
            array[i][j] = editText(array[i][j])
        end
    end
    return array
end

essays = editEssays(essays)
array = editParagraphs(array)

# Question 9
authorSet = ["HAMILTON", "MADISON", "JAY", "UNKNOWN", "ALL"]

function addParagraphsByAuthor(paragraphs, authors)
    essaysByAuthor = [Array{String}[], Array{String}[], Array{String}[], Array{String}[], Array{String}[]]
    for i in eachindex(authors)
        if(cmp(authors[i], "HAMILTON") == 0)
            hamilton = Array{String}(undef, 0)
            for j in eachindex(paragraphs[i])
                push!(hamilton, paragraphs[i][j])
            end
            push!(essaysByAuthor[1], hamilton)
        elseif(cmp(authors[i], "MADISON") == 0)
            madison = Array{String}(undef, 0)
            for j in eachindex(paragraphs[i])
                push!(madison, paragraphs[i][j])
            end
            push!(essaysByAuthor[2], madison)
        elseif(cmp(authors[i], "MADISON, WITH HAMILTON") == 0)
            hamilton = Array{String}(undef, 0)
            madison = Array{String}(undef, 0)
            for j in eachindex(paragraphs[i])
                push!(hamilton, paragraphs[i][j])
                push!(madison, paragraphs[i][j])
            end
            push!(essaysByAuthor[1], hamilton)
            push!(essaysByAuthor[2], madison)
        elseif(cmp(authors[i], "JAY") == 0)
            jay = Array{String}(undef, 0)
            for j in eachindex(paragraphs[i])
                push!(jay, paragraphs[i][j])
            end
            push!(essaysByAuthor[3], jay)
        else
            unknown = Array{String}(undef, 0)
            for j in eachindex(paragraphs[i])
                push!(unknown, paragraphs[i][j])
            end
            push!(essaysByAuthor[4], unknown)
        end
    end
    essaysByAuthor[5] = paragraphs
    return essaysByAuthor
end

paragraphsByAuthor = addParagraphsByAuthor(array, authors)

# Question 9a
function avgWordsPerEssay(authorParagraphs)
    wordCount = 0
    for i in eachindex(authorParagraphs)
        for j in eachindex(authorParagraphs[i])
            wordCount += (count(" ", authorParagraphs[i][j]) + 1)
        end
    end
    return wordCount / length(authorParagraphs)
end

for i in 1:5
    display("Number of words per essay by $(authorSet[i]): $(avgWordsPerEssay(paragraphsByAuthor[i]))")
end

function find_essays_by_author(essays, author)
    indices = findall(x -> x == author, authors)
    return essays[indices]
end

function average_word_count(essays)
    total_word_count = 0
    for essay in essays
        words = split(essay, r"\s+")
        total_word_count += length(words)
    end
    return total_word_count/length(essays)
end

function average_word_length(essays)
    total_word_length = 0
    total_word_count = 0
    for essay in essays
        words = split(essay, r"\s+")
        for word in words
            total_word_length += length(word)
        end
        total_word_count += length(words)
    end
    return total_word_length/total_word_count
end

function average_sentence_count(essays)
    total_sentence_count = 0
    for essay in essays
        sentences = split(essay, r"[.?!]")
        total_sentence_count += length(sentences)
    end
    return total_sentence_count/length(essays)
end

function average_sentence_per_paragraph(essays)
    total_sentence_count = 0
    total_paragraph_count = 0
    for essay in essays
        paragraphs = split(essay, "\n")
        total_paragraph_count += length(paragraphs)
        for paragraph in paragraphs
            sentences = split(paragraph, r"[.?!]")
            total_sentence_count += length(sentences)
        end
    end
    return total_sentence_count/total_paragraph_count
end

function unique_word_count(essays)
    word_dict = Dict()
    for essay in essays
        words = split(essay, r"\s+")
        for word in words
            if haskey(word_dict, word)
                word_dict[word] += 1
            else
                word_dict[word] = 1
            end
        end
    end
    return word_dict
end


#for author in authorSet
author  = "HAMILTON"
    author_essays = find_essays_by_author(essays, author)
    
    println(author, " wrote ", length(author_essays), " essays.")
    println("Average word count per essay: ", average_word_count(author_essays))
    println("Average word length per essay: ", average_word_length(author_essays))
    println("Average sentence count per essay: ", average_sentence_count(author_essays))
    println("Average sentence count per paragraph: ", average_sentence_per_paragraph(author_essays))
    word_dict = unique_word_count(author_essays)
    println("Unique words and their frequency:")
    for (word, count) in word_dict
        println(word, ": ", count)
    end

#end


# First, obtain the word count dictionaries for each author
madison_essays = find_essays_by_author(essays, "MADISON")
madison_words = unique_word_count(madison_essays)
hamilton_essays = find_essays_by_author(essays, "HAMILTON")
hamilton_words = unique_word_count(hamilton_essays)
jay_essays = find_essays_by_author(essays, "JAY")
jay_words = unique_word_count(jay_essays)

# Define the Score function
# Currently need to write into the loop above to save time
function Score(author_words, other1_words, other2_words, word)
    M = get(author_words, word, 0) / sum(values(author_words))
    H = get(other1_words, word, 0) / sum(values(other1_words))
    J = get(other2_words, word, 0) / sum(values(other2_words))
    return abs(M - H) + abs(M - J)
end

# Find the 10 words with the highest Score for each author
madison_scores = Dict()
for word in keys(madison_words)
    madison_scores[word] = Score(madison_words, hamilton_words, jay_words, word)
end
madison_top10 = sort(collect(madison_scores), by=x->x[2], rev=true)[1:10]

hamilton_scores = Dict()
for word in keys(hamilton_words)
    hamilton_scores[word] = Score(hamilton_words, madison_words, jay_words, word)
end
hamilton_top10 = sort(collect(hamilton_scores), by=x->x[2], rev=true)[1:10]

jay_scores = Dict()
for word in keys(jay_words)
    jay_scores[word] = Score(jay_words, madison_words, hamilton_words, word)
end
jay_top10 = sort(collect(jay_scores), by=x->x[2], rev=true)[1:10]

# Print the results
println("Top 10 words for Madison:")
for word in madison_top10
    println(word[1])
end

println("Top 10 words for Hamilton:")
for word in hamilton_top10
    println(word[1])
end

println("Top 10 words for Jay:")
for word in jay_top10
    println(word[1])
end

"Number of words per essay by HAMILTON: 2179.823529411765"

"Number of words per essay by MADISON: 2768.5714285714284"

"Number of words per essay by JAY: 1674.2"

"Number of words per essay by UNKNOWN: 1976.0"

"Number of words per essay by ALL: 2211.0823529411764"

...

1
contemplated.: 1
invading: 1
easy: 33
lastly: 2
aggressing: 1
conjuncture: 1
confidence: 34
uncandid: 1
riveting: 1
shape: 7
womb: 1
authority.: 26
conclave: 1
extremes: 6
preeminence: 4
obstruct: 4
country: 61
business: 17
unless: 24
inadequate.: 2
degrading: 1
dependence: 7
economy.: 4
the: 10178
lakes

: 1
links: 2
orbs: 1
urgent: 2
landholders: 1
technical: 5
intelligence: 6
hypothesis: 2
forced: 4
plentitude: 1
arm: 5
read: 2
exceptions: 17
understand: 8
agents.: 1
requires: 9
breaking: 1
"equity".: 1
disregard: 5
secret: 4
greatest: 18
sessions: 2
vassals: 4
legerdemain.: 1
discrimination: 6
uncommonly: 1
achaean: 1
splendid: 1
innumerable.: 1
acquisition.: 1
trade?: 1
dread: 5
usefully: 4
designate: 2
apply.: 1
determinable: 7
handmaid: 1
participated.: 1
mismanagement: 4
unfriendly: 9
stubborn: 1
injuriously: 1
counsel: 1
admissible.: 1
embracing: 1
proves: 8
grounded: 1
vanity: 3
place.: 5
hereditary: 9
enthusiasm: 3
station.: 2
dictates: 9
largest: 2
wanted: 2
institutions.: 4
sedate: 1
ban: 1
deserve: 9
persuasion: 3
logic: 1
group.: 1
husbandman: 1
merchant: 10
year: 4
ministers: 11
regulations: 23
comparative: 5
gross: 4
broached: 1
proper.: 1
forbid.: 2
hard: 1
residence?: 1
distribution.: 1
mislead: 4
left: 16
conjunctures: 4
neglecting: 1
prudent: 6
respectable: 8
repeat

: 6
efficiency: 3
scanty: 2
commit: 9
obviate: 9
triple: 2
prejudice.: 1
perusal: 1
first.: 4
exclusively: 7
alternate: 1
course: 80
wills: 1
utility.: 2
dissimilitude: 2
watchfulness: 1
prerogatives: 4
young: 1
umbrage: 1
constitution": 2
europe: 16
legislation.: 1
hold: 20
maladies: 1
republican: 28
conditions: 2
circles: 3
fairsounding: 1
writers.: 1
confining: 5
precious: 1
buildings: 1
rectitude.: 1
respectively: 1
pope

: 1
fourteen: 1
examined: 6
rotation: 3
discourages: 1
avowed: 1
abilities.: 1
observance: 4
provident: 1
impracticability.: 1
alliances: 4
training: 1
destroying: 1
subvert: 4
secure: 15
agents: 3
resembles: 1
trembling: 1
impediments: 5
extremities: 1
profess: 3
despicable: 3
undergoes: 1
affinity: 4
feared: 3
declaring: 3
kindred: 2
plainest: 3
herculean: 1
impairs: 1
itself.: 10
on: 374
inexpedient: 1
beforehand: 3
requisite: 34
amounts: 5
speaking: 3
reasoner: 2
exported: 1
infuse: 2
world.: 7
latent: 1
regulator: 1
overmatch: 3
degrades: 1
emoluments.: 1
corrupting: 1
vulnerable.: 1
measure.: 1
expedient.: 1
ingredient: 5
quantity: 11
convincing: 1
preoccupied: 1
yet: 69
chastise: 1
republican?: 1
larger: 9
french: 2
disorder: 4
efficacious: 4
accomplished?: 1
fourth: 8
blending: 1
welltrained: 1
attest: 1
pursuit: 5
"exclusive: 1
disappointment: 1
adding: 1
place: 65
poetic: 1
atone: 2
small: 29
elapsed: 1
deed: 3
bound: 10
expect: 14
bears: 3
attract: 2
selected: 6
arithmetic.:


Top 10 words for Madison:
the
and
of
to
a
would
on
is
by
they
Top 10 words for Hamilton:
the
and
of
to
a
in
upon
by
on
they
Top 10 words for Jay:
the
and
of
a
they
is
or
their
which
to


In [ ]:
# Tester
text = read("federalistpapers.txt", String)
essays = split(text, r"FEDERALIST No.\ \d+")
popfirst!(essays)
println(length(essays))
authors = Array{String}(undef, length(essays))
for i in eachindex(essays)
    essayParts = split(essays[i], "PUBLIUS")
    essayParts = split(essayParts[1], "To the People of the State of New York")
    firstPart = split(essayParts[1], "\r\n\r\n")
    if(i in 49:58 || i in 62:63)
        authors[i] = "Unknown"
    else
        authors[i] = replace(firstPart[end - 1], "\r\n" => "")
    end
    secondPart = essayParts[2]
    essays[i] = string(secondPart[6:end], "PUBLIUS")
end

# println(essays[1])

In [ ]:
# Minh's implementation

# Note: the number of the essay will be index no.

# Break it up into the individual essays. Each essay starts with "Federalist No. n." Within each essay you can skip everything up to and including "To the People of the State of New York:" and the footnotes after "PUBLIUS." You can also skip everything up to the start of the first essay and everything after the end of the last essay. (3 points)
text = read("federalistpapers.txt", String)
essays = split(text, r"FEDERALIST No.\ \d+")
popfirst!(essays)

from = "To the People of the State of New York:"
to = "PUBLIUS" 

# Loop over each essay and extract the text in the essays
for (i, essay) in enumerate(essays)

    search_from = findfirst(from, essay)
    search_to = findfirst(to, essay)

    # Find the indices of from and to in the essay string
    start_index = search_from[1] + length(from)
    end_index = search_to[1] - 1
    
    # Extract the text between from and to
    extracted_text = essay[start_index:end_index]

    # Remove trailing new lines in the start and the end of the needed text
    extracted_text = rstrip(extracted_text) # somehow this does not remove the last line of the last paragraph, but it create a concrete pattern for all paragraphs (to have one new line at the end), so I guess it works.
    extracted_text = lstrip(extracted_text)

    # Print the extracted text
    # println(extracted_text)
    # println(i)

    # Replace with only the needed text from each essay
    essays[i] = extracted_text 
    
end
# println(typeof(essays[1]))
println(essays[1])

